# Generate text file with corrected documents for UA-GEC validation set

In [1]:
!pip install -q nltk
!pip install -q toolz

In [2]:
from difflib import SequenceMatcher
import nltk
import re
import toolz
import gc
import os
import re

nltk.download('punkt')  # Download the necessary resources for sentence tokenization

from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Helper methods

In [3]:
def highlight_changes(text1, text2):
    # Tokenize the texts into words
    words1 = re.findall(r'\w+|[^\w\s]', text1)
    words2 = re.findall(r'\w+|[^\w\s]', text2)


    # Find the unique words present in both texts
    all_words = set(words1 + words2)

    # Initialize a SequenceMatcher object
    matcher = SequenceMatcher(None, words1, words2)

    # Get the differences
    diff = matcher.get_opcodes()

    highlighted_text = []

    for op, start1, end1, start2, end2 in diff:
        if op == 'equal':
            # No change, just append the words as is
            highlighted_text.extend(words1[start1:end1])
        elif op == 'delete':
            # Word(s) removed, highlight with red
            for word in words1[start1:end1]:
                word = '\u0336'.join(word) + '\u0336'
                highlighted_text.append('\033[91m\033[1m' + word + '\033[0m')
        elif op == 'insert':
            # Word(s) added, highlight with green
            for word in words2[start2:end2]:
                highlighted_text.append('\033[92m\033[1m' + word + '\033[0m')
        elif op == 'replace':
            # Word(s) replaced, highlight with yellow
            for word in words2[start2:end2]:
                highlighted_text.append('\033[93m\033[1m' + word + '\033[0m')

    return ' '.join(highlighted_text)

def generate_original_corrected_texts(original_text, corrected_text):
    # Split the original and corrected texts
    original_words = original_text.split()
    corrected_words = corrected_text.split()

    # Initialize empty lists for marked original and corrected texts
    marked_original_text = []
    marked_corrected_text = []

    # Track words from the original text that were removed
    removed_words = set(original_words) - set(corrected_words)

    # Track words from the corrected text that were added
    added_words = set(corrected_words) - set(original_words)

    # Mark removed words in the original text as red
    for word in original_words:
        if word in removed_words:
            marked_original_text.append('\033[91m\033[1m' + word + '\033[0m')
        else:
            marked_original_text.append(word)

    # Mark added words in the corrected text as green
    for word in corrected_words:
        if word in added_words:
            marked_corrected_text.append('\033[92m\033[1m' + word + '\033[0m')
        else:
            marked_corrected_text.append(word)

    return (' '.join(marked_original_text), ' '.join(marked_corrected_text))


def print_text_comparison(original, generated):
    highlighted_text = highlight_changes(original, generated)

    original_text, corrected_text, = generate_original_corrected_texts(
        original_text=original, 
        corrected_text=generated
    )

    print(f"Original Text:\n{original_text}")

    print(f"\nCorrected Text:\n{corrected_text}")

    print(f"\nChanges comparison:\n{highlighted_text}")

### Item #0006 from [UA-GEC validation set](https://github.com/grammarly/ua-gec/tree/main/data/gec-only/test)

In [4]:
source='''
Привіт Настя
А діти в садочку після сніданку гуляють?
Щось ми сьогодні не встигаємо до дев'ятої привести
Дякую за інформацію! ми з Надією саме вийшли з дому
Ми прийшли, але діти ще не гуляють.
Сказали нам почекати поки група вийде.
Надя щось не в захваті, каже: "не піду!"
Ох, все дуже заплутано.
Ми у вересні подивимось який в Уляни буде розклад в школі й по гуртках, і як воно буде Надії в садочку, і вирішимо як краще ходити.
В поки що вересень будем повну оплату робити.
У нас тут ще й в школі незрозуміло.
Кажуть що діти будуть або через день займатись, або через тиждень.
То ми моде ще й Уляну будемо в совенята через день відправляти
Всі батьки в шоці ))
Отож, я й сама ні в чому не впевнена
А мені здається що "все" вже не будуть закривати.
Люди всі втомилися, всім байдуже
Я збираюся забрати Надію раніше, бо мені сказали що на цьому тижні тільки на прогулянки будут її брати, тобто до обіду.
Я вже поруч з садком в кафе сиджу, розмовляю з замовниками, ніяк не можу закінчити розмову )))
Потім я їй обіцяла зайти в арома каву по какао
А потім додому підемо
Краще наступним разом, бо не знаю чи буде Надія в доброму гуморі
'''

In [5]:
target='''
Привіт, Насте.
А діти в садочку після сніданку гуляють?
Щось ми сьогодні не встигаємо до дев'ятої привести.
Дякую за інформацію! Ми з Надією саме вийшли з дому.
Ми прийшли, але діти ще не гуляють.
Сказали нам почекати, поки група вийде.
Надя щось не в захваті, каже: "Не піду!"
Ох, усе дуже заплутано.
Ми у вересні подивимось, який в Уляни буде розклад у школі й по гуртках, і як воно буде Надії в садочку, і вирішимо, як краще ходити.
Поки що вересні будем повну оплату робити.
У нас тут ще й у школі незрозуміло.
Кажуть, що діти будуть або через день займатись, або через тиждень.
То ми, може, ще й Уляну будемо в совенята через день відправляти
Всі батьки в шоці ))
Отож, я й сама ні в чому не впевнена
А мені здається, що "все" вже не будуть закривати.
Люди всі втомилися, всім байдуже.
Я збираюся забрати Надію раніше, бо мені сказали що на цьому тижні тільки на прогулянки будуть її брати, тобто до обіду.
Я вже поруч зі садком в кафе сиджу, розмовляю зі замовниками, ніяк не можу закінчити розмову )))
Потім я їй обіцяла зайти в "Арома Каву" по какао.
А потім додому підемо.
Краще наступним разом, бо не знаю, чи буде Надія в доброму гуморі.
'''

## Perfect correction

In [6]:
print_text_comparison(source, target)

Original Text:
Привіт Настя А діти в садочку після сніданку гуляють? Щось ми сьогодні не встигаємо до дев'ятої привести Дякую за інформацію! ми з Надією саме вийшли з дому Ми прийшли, але діти ще не гуляють. Сказали нам почекати поки група вийде. Надя щось не в захваті, каже: "не піду!" Ох, все дуже заплутано. Ми у вересні подивимось який в Уляни буде розклад в школі й по гуртках, і як воно буде Надії в садочку, і вирішимо як краще ходити. В поки що вересень будем повну оплату робити. У нас тут ще й в школі незрозуміло. Кажуть що діти будуть або через день займатись, або через тиждень. То ми моде ще й Уляну будемо в совенята через день відправляти Всі батьки в шоці )) Отож, я й сама ні в чому не впевнена А мені здається що "все" вже не будуть закривати. Люди всі втомилися, всім байдуже Я збираюся забрати Надію раніше, бо мені сказали що на цьому тижні тільки на прогулянки будут її брати, тобто до обіду. Я вже поруч з садком в кафе сиджу, розмовляю з замовниками, ніяк не можу закінчити 

## Generated output by zero-shot Mistral 7B model

In [7]:
generated_zero_shot='''
 Привіт Настя

А діти в садочці після сніданку гуляють?
Щось ми сьогодни не встигаємо до дев'ятої привести.
Дякую за інформацію!
Ми з Надією саме вийшли з дому.
Ми прийшли, але діти ще не гуляють.
Сказали нам почекати, поки група вийде.
Надя щось не в захваті, каже: "Не піду!"
Ох, все дуже заплутано.
Ми у вересні подивимося, яка в Уляни буде розклад в школі, і як воно буде Надії в садочці, і вирішим, як краще ходити.
В поки що вересень будемо повну оплату робити.
У нас тут ще й в школі незрозуміло.
Вони кажуть, що діти будуть або завтра займатися, або наступного тижня.
Тому ми можемо ще й Уляну наступного дні відправляти до совенята.
Всі батьки в шоці :)

Отож, я сама ні в чому не впевнена

А мені здається, що "все" вже не будуть закривати.
Люди всі втомилися, всім байдуже.
Я збираюся забрати Надію раніше, бо мені сказали, що на цьому тижні тільки на прогулянки будуть її брати, тобто до обіду.
Я вже поруч з садком у кафе сижу, розмовляю з замовниками, ніяк не можу закінчити розмову :)

Потім я їй обіцяла пойти до арома кави по какао.
А потім додому підемо.
Краще наступним разом, бо не знаю, чи буде Надія в доброму гуморі.
'''

In [8]:
print_text_comparison(source, generated_zero_shot)

Original Text:
Привіт Настя А діти в садочку після сніданку гуляють? Щось ми сьогодні не встигаємо до дев'ятої привести Дякую за інформацію! ми з Надією саме вийшли з дому Ми прийшли, але діти ще не гуляють. Сказали нам почекати поки група вийде. Надя щось не в захваті, каже: "не піду!" Ох, все дуже заплутано. Ми у вересні подивимось який в Уляни буде розклад в школі й по гуртках, і як воно буде Надії в садочку, і вирішимо як краще ходити. В поки що вересень будем повну оплату робити. У нас тут ще й в школі незрозуміло. Кажуть що діти будуть або через день займатись, або через тиждень. То ми моде ще й Уляну будемо в совенята через день відправляти Всі батьки в шоці )) Отож, я й сама ні в чому не впевнена А мені здається що "все" вже не будуть закривати. Люди всі втомилися, всім байдуже Я збираюся забрати Надію раніше, бо мені сказали що на цьому тижні тільки на прогулянки будут її брати, тобто до обіду. Я вже поруч з садком в кафе сиджу, розмовляю з замовниками, ніяк не можу закінчити 

## Generated output by LoRA fine-tuned with r=4 on 500 samples from train set

In [9]:
generated_r4_500='''
Привіт, Настя.
А діти в садочку після сніданку гуляють?
Щось ми сьогодні не встигаємо до дев'ятої привести.
Дякую за інформацію!
Ми з Надією саме вийшли з дому.
Ми прийшли, але діти ще не гуляють.
Сказали нам почекати, поки група вийде.
Надя щось не в захваті, каже: "не піду!
"Ох, все дуже заплутано.
Ми у вересні подивимось, який в Уляни буде розклад в школі й по гуртках, і як воно буде Надії в садочку, і вирішимо, як краще ходити.
В поки що вересень будемо повну оплату робити.
У нас тут ще й в школі незрозуміло.
Кажуть, що діти будуть або через день займатись, або через тиждень.
То ми моде ще й Уляну будемо в совенята через день відправляти.
Всі батьки в шоці )) Отож, я й сама ні в чому не впевнена.
А мені здається, що "все" вже не будуть закривати.
Люди всі втомилися, всім байдуже.
Я збираюся забрати Надію раніше, бо мені сказали, що на цьому тижні тільки на прогулянки будуть її брати, тобто до обіду.
Я вже поруч із садком у кафе сиджу, розмовляю з замовниками, ніяк не можу закінчити розмову ))) Потім я їй обіцяла зайти в арома каву по какао.
А потім додому підемо.
Краще наступним разом, бо не знаю, чи буде Надія в доброму гуморі.'''

In [10]:
print_text_comparison(source, generated_r4_500)

Original Text:
Привіт Настя А діти в садочку після сніданку гуляють? Щось ми сьогодні не встигаємо до дев'ятої привести Дякую за інформацію! ми з Надією саме вийшли з дому Ми прийшли, але діти ще не гуляють. Сказали нам почекати поки група вийде. Надя щось не в захваті, каже: "не піду!" Ох, все дуже заплутано. Ми у вересні подивимось який в Уляни буде розклад в школі й по гуртках, і як воно буде Надії в садочку, і вирішимо як краще ходити. В поки що вересень будем повну оплату робити. У нас тут ще й в школі незрозуміло. Кажуть що діти будуть або через день займатись, або через тиждень. То ми моде ще й Уляну будемо в совенята через день відправляти Всі батьки в шоці )) Отож, я й сама ні в чому не впевнена А мені здається що "все" вже не будуть закривати. Люди всі втомилися, всім байдуже Я збираюся забрати Надію раніше, бо мені сказали що на цьому тижні тільки на прогулянки будут її брати, тобто до обіду. Я вже поруч з садком в кафе сиджу, розмовляю з замовниками, ніяк не можу закінчити 

## Generated output by LoRA fine-tuned with r=4 on 1500 samples from train set¶

In [11]:
generated_r4_1500='''
Привіт, Настю.
А діти в садочку після сніданку гуляють?
Щось ми сьогодні не встигаємо до дев'ятої привести.
Дякую за інформацію!
Ми з Надією саме вийшли з дому.
'''

In [12]:
print_text_comparison(source, generated_r4_1500)

Original Text:
Привіт Настя А діти в садочку після сніданку гуляють? Щось ми сьогодні не встигаємо до дев'ятої привести Дякую за інформацію! ми з Надією саме вийшли з дому Ми прийшли, але діти ще не гуляють. Сказали нам почекати поки група вийде. Надя щось не в захваті, каже: "не піду!" Ох, все дуже заплутано. Ми у вересні подивимось який в Уляни буде розклад в школі й по гуртках, і як воно буде Надії в садочку, і вирішимо як краще ходити. В поки що вересень будем повну оплату робити. У нас тут ще й в школі незрозуміло. Кажуть що діти будуть або через день займатись, або через тиждень. То ми моде ще й Уляну будемо в совенята через день відправляти Всі батьки в шоці )) Отож, я й сама ні в чому не впевнена А мені здається що "все" вже не будуть закривати. Люди всі втомилися, всім байдуже Я збираюся забрати Надію раніше, бо мені сказали що на цьому тижні тільки на прогулянки будут її брати, тобто до обіду. Я вже поруч з садком в кафе сиджу, розмовляю з замовниками, ніяк не можу закінчити 

## Generated output by LoRA fine-tuned with r=8 on 1000 samples from train set

In [13]:
generated_r8_1000='''
Привіт, Настю.
А діти в садочку після сніданку гуляють?
Щось ми сьогодні не встигаємо до дев'ятої привести.
Дякую за інформацію!
Ми з Надією саме вийшли з дому.
'''

In [14]:
print_text_comparison(source, generated_r8_1000)

Original Text:
Привіт Настя А діти в садочку після сніданку гуляють? Щось ми сьогодні не встигаємо до дев'ятої привести Дякую за інформацію! ми з Надією саме вийшли з дому Ми прийшли, але діти ще не гуляють. Сказали нам почекати поки група вийде. Надя щось не в захваті, каже: "не піду!" Ох, все дуже заплутано. Ми у вересні подивимось який в Уляни буде розклад в школі й по гуртках, і як воно буде Надії в садочку, і вирішимо як краще ходити. В поки що вересень будем повну оплату робити. У нас тут ще й в школі незрозуміло. Кажуть що діти будуть або через день займатись, або через тиждень. То ми моде ще й Уляну будемо в совенята через день відправляти Всі батьки в шоці )) Отож, я й сама ні в чому не впевнена А мені здається що "все" вже не будуть закривати. Люди всі втомилися, всім байдуже Я збираюся забрати Надію раніше, бо мені сказали що на цьому тижні тільки на прогулянки будут її брати, тобто до обіду. Я вже поруч з садком в кафе сиджу, розмовляю з замовниками, ніяк не можу закінчити 